In [2]:
import xarray as xr
import glob
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xesmf as xe
import datetime as dt
import pandas as pd

In [260]:
files = glob.glob('/scratch/averstyn/S5P/co/oct2022_cropped/*.nc')
files

['/scratch/averstyn/S5P/co/oct2022_cropped/20221001T190949_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221002T203209_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221003T201300_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221004T195351_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221005T193442_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221006T191533_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221007T203753_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221008T201844_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221009T195935_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221011T192117_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221012T204337_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221014T200519_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221015T194610_cropped.nc',
 '/scratch/averstyn/S5P/co/oct2022_cropped/20221016T192700_cropp

In [4]:
new_lat=np.arange(-90,90.5,0.5)
new_lon=np.arange(0,360.1,0.1)

ds_out = xr.Dataset({'lat': (['lat'], new_lat),
                    'lon': (['lon'], new_lon)
                    })
ds_out

<xarray.Dataset>
Dimensions:  (lat: 361, lon: 3601)
Coordinates:
  * lat      (lat) float64 -90.0 -89.5 -89.0 -88.5 -88.0 ... 88.5 89.0 89.5 90.0
  * lon      (lon) float64 0.0 0.1 0.2 0.3 0.4 ... 359.6 359.7 359.8 359.9 360.0
Data variables:
    *empty*

In [324]:
var = 'carbonmonoxide_total_column'
file = files[24]
ds = xr.open_dataset(file)
ds2 = ds.sel(time='2022-10-30')
#regridder = xe.Regridder(ds2,ds_out,'bilinear')
#dr_out = regridder(ds[var]).to_dataset(name=var)
#dr_out.reset_coords()
#dr_out
#dr_out.to_netcdf('/scratch/averstyn/final_s5p/20200801_no2.nc')
ds

<xarray.Dataset>
Dimensions:                                (scanline: 501, ground_pixel: 215,
                                            time: 1, corner: 4, layer: 50)
Coordinates:
  * scanline                               (scanline) float64 2.9e+03 ... 3.4...
  * ground_pixel                           (ground_pixel) float64 0.0 ... 214.0
  * time                                   (time) datetime64[ns] 2022-10-30
  * corner                                 (corner) float64 0.0 1.0 2.0 3.0
  * layer                                  (layer) float32 4.95e+04 ... 500.0
    latitude                               (time, scanline, ground_pixel) float32 ...
    longitude                              (time, scanline, ground_pixel) float32 ...
Data variables:
    delta_time                             (time, scanline) datetime64[ns] ...
    qa_value                               (time, scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column            (time, scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column_precision  (time, scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column_corrected  (time, scanline, ground_pixel) float32 ...

In [325]:
ds2

<xarray.Dataset>
Dimensions:                                (scanline: 501, ground_pixel: 215,
                                            corner: 4, layer: 50)
Coordinates:
  * scanline                               (scanline) float64 2.9e+03 ... 3.4...
  * ground_pixel                           (ground_pixel) float64 0.0 ... 214.0
    time                                   datetime64[ns] 2022-10-30
  * corner                                 (corner) float64 0.0 1.0 2.0 3.0
  * layer                                  (layer) float32 4.95e+04 ... 500.0
    latitude                               (scanline, ground_pixel) float32 ...
    longitude                              (scanline, ground_pixel) float32 ...
Data variables:
    delta_time                             (scanline) datetime64[ns] ...
    qa_value                               (scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column            (scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column_precision  (scanline, ground_pixel) float32 ...
    carbonmonoxide_total_column_corrected  (scanline, ground_pixel) float32 ...

In [326]:
regridder = xe.Regridder(ds2,ds_out,'bilinear')
dr_out = regridder(ds[var]).to_dataset(name=var)
dr_out.reset_coords()
dr_out

<xarray.Dataset>
Dimensions:                      (time: 1, lat: 361, lon: 3601)
Coordinates:
  * time                         (time) datetime64[ns] 2022-10-30
  * lat                          (lat) float64 -90.0 -89.5 -89.0 ... 89.5 90.0
  * lon                          (lon) float64 0.0 0.1 0.2 ... 359.8 359.9 360.0
Data variables:
    carbonmonoxide_total_column  (time, lat, lon) float32 0.0 0.0 ... 0.0 0.0

In [327]:
dr_out.to_netcdf('/home/averstyn/final_s5p/co/20221030_co.nc')

In [ ]:
def makeds(filenum,day):
    var = 'nitrogendioxide_tropospheric_column'
    file = files[filenum]
    ds = xr.open_dataset(file)
    ds2 = ds.sel(time='2019-09-0'+str(day))
    regridder = xe.Regridder(ds2,ds_out,'bilinear')
    dr_out = regridder(ds[var]).to_dataset(name=var)
    dr_out.reset_coords()
    return dr_out

In [ ]:
def dstocdf(ds,day):
    ds.to_netcdf('/scratch/averstyn/final_s5p/2019090'+str(day)+'_co.nc')

In [ ]:
ds = makeds(3,4)
dstocdf(ds,4)

In [ ]:
ds = xr.open_dataset(files[1])
ds

In [261]:
var = 'carbonmonoxide_total_column'
for i in range(9):
    file = files[i]
    ds = xr.open_dataset(file)
    ds2 = ds.sel(time='2022-10-0'+str(i+1))
    regridder = xe.Regridder(ds2,ds_out,'bilinear')
    dr_out = regridder(ds[var]).to_dataset(name=var)
    dr_out.reset_coords()
    dr_out.to_netcdf('/home/averstyn/final_s5p/co/2022100'+str(i+1)+'_co.nc')

In [337]:
var = 'carbonmonoxide_total_column'
for i in range(5,14):
    file = files[i]
    ds = xr.open_dataset(file)
    ds2 = ds.sel(time='2021-10-'+str(i+7))
    regridder = xe.Regridder(ds2,ds_out,'bilinear')
    dr_out = regridder(ds[var]).to_dataset(name=var)
    dr_out.reset_coords()
    dr_out.to_netcdf('/home/averstyn/final_s5p/co/202010'+str(i+1)+'_co.nc')